In [ ]:
import pandas as pd
import numpy as np
import math
import subprocess
from datetime import date, time, datetime, timedelta
from IPython.display import display
import calendar
import bandit.main as bd

In [ ]:
def df_search(df_in, cohort):
    if not cohort in df_in:
        return df_in

    df_class = bd.df_bandit_class(df_in, cohort)
    display(df_class)
    next_class = df_class.iloc[0, 0]
#     if df_in['Complete'].isnull().sum() > 0:
#         df_filtered = df_in[df_in[cohort] == next_class]
#     else:
#         df_filtered = df_in
        
    df_filtered = df_in[df_in[cohort] == next_class]
    return df_filtered

In [ ]:
def date_filter(df):
    currDay = calendar.day_name[currDate.weekday()]
    df_today = df
    df_today['weekday'] = df_today['Date'].dt.day_name()
    df_today = df_today[df_today['weekday'] == currDay]
    today_sum = df_today['Complete'].notnull().sum()
    # df_today = df_today.drop(['weekday'],axis=1)

    if today_sum >= 10:
        return df_today
    return df

In [ ]:
def rate_calc(df):
    pedal_time = df['Goal'] * df['seconds']
    rest_time = (df['Goal'] - 1) * 15
    total_time = pedal_time + rest_time
    rate = pedal_time / total_time
    score = pedal_time * rate
    return int(score)

In [ ]:
currDate = datetime.today()
goal_list = [y for y in range(1,26)]
df = pd.DataFrame({"Goal":goal_list})
df['Date'] = currDate
df['Complete'] = np.nan
df = df[['Date', 'Goal', 'Complete']]

In [ ]:
df_excel = pd.read_excel(io='GoalSetter2.xlsx')
df_excel = date_filter(df_excel)
foo = df_excel.groupby('Goal').count()
foo['remove'] = np.where(foo['Complete'] > 0, True, False)
foo = foo[['remove']]
df = df.merge(foo,on='Goal',how='left')
df = df[df['remove'] != True].drop(['remove'],axis=1)
df

In [ ]:
subprocess.Popen('GoalSetter2.xlsx', shell=True)

In [ ]:
if len(df) > 0:
    df = df.append(df_excel).reset_index(drop=True)
else:
    df = df_excel

In [ ]:
df = bd.reduce(df,'Goal')

In [ ]:
df.head(10)

In [ ]:
df['Score'] = np.where(df['Complete'] >= df['Goal'], df['Goal']**1.2, df['Complete'] / 5)

In [ ]:
df_quality = bd.get_quality(df['Score'])

In [ ]:
val_df = bd.set_cohorts(df, 'Goal')
df = df.merge(val_df, on='Goal')

In [ ]:
# todo: move date filter to before reduce and pruning
currDay = calendar.day_name[currDate.weekday()]
df_today = df
df_today['weekday'] = df_today['Date'].dt.day_name()
df_today = df_today[df_today['weekday'] == currDay]
today_sum = df_today['Complete'].notnull().sum()
if today_sum >= 10:
    df = df_today

In [ ]:
df.sort_values('Score', ascending=False).head(20)

In [ ]:
df.sort_values('Date', ascending=True).tail(10)

In [ ]:
df_quality = bd.get_quality(df['Score'])
df = df.join(df_quality)

In [ ]:
df_step1 = df_search(df,'cohort1')
df_step1

In [ ]:
df_step2 = df_search(df_step1,'cohort2')
df_step2

In [ ]:
df_step3 = df_search(df_step2,'cohort3')
df_step3.head(10)

In [ ]:
bd.df_bandit_class(df_step3, 'Goal').head(10)